# Create the PPL correlations

In [ ]:
import pickle as pkl
import numpy as np
from scipy.stats import pearsonr

# name_file = 'Perturbed_tweets_test_english_val.tsv'
name_file = 'Eurotweets_English_val_without_line_return.tsv_clean_test'

path_data = '/home/barriva/data/PPL_Positivity/'
path_dump_perturbed = path_data + 'Perturbed_' + name_file #+ '.pkl_ERROR'

with open(path_dump_perturbed, 'rb') as fp:
    perturbed_X_text = pkl.load(fp)

In [4]:
with open(path_dump_perturbed + '_PPL.pkl', 'rb') as f:
    list_PPL = pkl.load(f)

list_gender = ['male', 'female']

list_countries = []

for k in list_PPL.keys():
    try:
        if len(list_PPL[k]['female']):
            list_countries.append(k)
    except:
        pass
list_countries

['France', 'United_Kingdom', 'Ireland', 'Spain', 'Germany', 'Italy', 'Morocco']

In [5]:
proba = {ct: {} for ct in list_countries}
for country in list_countries:
    for gender in list_gender:
        with open(path_data + 'Proba_Perturbed_%s_%s_%s.pkl'%(name_file, country, gender), 'rb') as f:
            proba[country][gender] = pkl.load(f)

with open(path_data + 'ProbaIni_Perturbed_%s.pkl'%name_file, 'rb') as f:
    proba_ini = pkl.load(f)
    
n_counterfactuals = len(proba[country][gender]) // len(proba_ini)
proba_ini = np.repeat(proba_ini, n_counterfactuals, axis=0)
PPL_ini = np.repeat(list_PPL['Original'], n_counterfactuals)

In [6]:
label2idx = {'positive': 1, 'negative':0, 'neutral': 2}

proba_counter = { k : {ct : {gender: proba[ct][gender][:,l] for gender in list_gender} for ct in list_countries} for k,l in label2idx.items()}
proba_counter_without_offset = {v: {ct : {gender: proba_counter[v][ct][gender] - proba_ini[:,label2idx[v]] for gender in list_gender} for ct in list_countries} for v in proba_counter.keys()}
proba_counter_relative_v0 = {v: {ct : {gender: proba_counter_without_offset[v][ct][gender] - proba_ini[:,label2idx[v]] for gender in list_gender} for ct in list_countries} for v in proba_counter.keys()}
proba_counter_relative= {v: {ct : {gender: proba_counter_without_offset[v][ct][gender]/proba_ini[:,label2idx[v]] for gender in list_gender} for ct in list_countries} for v in proba_counter.keys()}

proba_counter['delta'] = {ct : {gender: proba[ct][gender][:,label2idx['positive']] - proba[ct][gender][:,label2idx['negative']] for gender in list_gender} for ct in list_countries}
proba_counter_without_offset['delta'] = {ct : {gender: proba_counter['delta'][ct][gender] - (proba_ini[:,label2idx['positive']] - proba_ini[:,label2idx['negative']]) for gender in list_gender} for ct in list_countries}
proba_counter_relative['delta'] = {ct : {gender: proba_counter_without_offset['delta'][ct][gender]/((proba_ini[:,label2idx['positive']] - proba_ini[:,label2idx['negative']])) for gender in list_gender} for ct in list_countries}

In [7]:
list_PPL_without_offset = {ct : {gender: list_PPL[ct][gender] - PPL_ini for gender in list_gender} for ct in list_countries}
list_PPL_relative = {ct : {gender: (list_PPL[ct][gender] - PPL_ini)/PPL_ini for gender in list_gender} for ct in list_countries}

# All at once 

In [23]:
proba_string = 'positive'
for country in list_countries:
    for gender in list_gender:
        print(country, gender, pearsonr(proba_counter[proba_string][country][gender], 
            list_PPL[country][gender]))

print('\nDELTA + PPL NO OFFSET')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_without_offset[proba_string][country][gender], 
            list_PPL_without_offset[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nPPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)

print('\nDELTA + PPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA RELATIVE PPL NO OFFSET')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL_without_offset[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA NO OFFSET + PPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_without_offset[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)

France male (0.1998123564079567, 1.4109697311948025e-230)
France female (0.21008167050059434, 3.255884423509013e-255)
United_Kingdom male (0.19237340891958843, 1.45627621203059e-213)
United_Kingdom female (0.19989398360730304, 9.098334458462982e-231)
Ireland male (0.18648332130056666, 1.3895819410979882e-200)
Ireland female (0.1920406378668042, 8.092779179264197e-213)
Spain male (0.18157789875412522, 4.201251598379549e-190)
Spain female (0.19695071937924352, 5.981100669047955e-224)
Germany male (0.1811279612816839, 3.713265543345987e-189)
Germany female (0.19810544323796386, 1.303107676330949e-226)
Italy male (0.19279867164917244, 1.6193645793719052e-214)
Italy female (0.20532141052836955, 1.2577039422461071e-243)
Morocco male (0.175093582857597, 1.0493692781984791e-176)
Morocco female (0.1713534198828672, 3.263677735353047e-169)

DELTA + PPL NO OFFSET
France male 4.42 1.2355901257342042e-12
France female 2.95 2.181564038270941e-06
United_Kingdom male 5.89 2.905730827908323e-21
United_

In [24]:
proba_string = 'negative'
for country in list_countries:
    for gender in list_gender:
        print(country, gender, pearsonr(proba_counter[proba_string][country][gender], 
            list_PPL[country][gender]))

print('\nDELTA + PPL NO OFFSET')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_without_offset[proba_string][country][gender], 
            list_PPL_without_offset[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nPPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)

print('\nDELTA + PPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA RELATIVE PPL NO OFFSET')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_relative[proba_string][country][gender], 
            list_PPL_without_offset[country][gender])
        print(country, gender, np.round(100*cor, 2), p)
        
print('\nDELTA NO OFFSET + PPL RELATIVE')
for country in list_countries:
    for gender in list_gender:
        cor, p = pearsonr(proba_counter_without_offset[proba_string][country][gender], 
            list_PPL_relative[country][gender])
        print(country, gender, np.round(100*cor, 2), p)

France male (-0.12190418403871175, 5.439153985706298e-86)
France female (-0.13149809418291575, 7.264750002041925e-100)
United_Kingdom male (-0.11523787790402788, 5.522516650478407e-77)
United_Kingdom female (-0.1253007296766252, 8.851516974019439e-91)
Ireland male (-0.12438554366420676, 1.7805525735051127e-89)
Ireland female (-0.12552125919662827, 4.279974735930971e-91)
Spain male (-0.10772068846722185, 1.8991410691355124e-67)
Spain female (-0.12352877361174193, 2.89733472411306e-88)
Germany male (-0.11118523975438135, 9.220999707250718e-72)
Germany female (-0.12430992551049734, 2.2794080556528674e-89)
Italy male (-0.12205524975915526, 3.352924271024014e-86)
Italy female (-0.13503706491662934, 2.951655389913313e-105)
Morocco male (-0.09813558057351388, 3.0967068148943725e-56)
Morocco female (-0.10614684764728728, 1.5570522460639926e-65)

DELTA + PPL NO OFFSET
France male 8.18 1.530555762326581e-39
France female 6.87 2.2854696657138976e-28
United_Kingdom male 15.03 3.2856882187117775e-1

## Sentence by sentence 

Equivalent to standardizing each values at the exemple level 

In [25]:
for country in list_countries:
    for gender in list_gender:
        sum_pears = []
        for k in range(n_counterfactuals):
            pears, p_val = pearsonr(rescale(proba_counter['positive'][country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals]), 
                    rescale(list_PPL[country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals])
                   )
            sum_pears.append(pears)
        print(country, gender, np.round(100*np.mean(sum_pears), 2), np.round(np.mean(p_val), 4))

France male 20.71 0.0
France female 9.16 0.0001
United_Kingdom male 6.78 0.0331
United_Kingdom female 15.34 0.0004
Ireland male 13.42 0.0101
Ireland female 6.41 0.0001
Spain male 6.36 0.0005
Spain female 5.91 0.0
Germany male 19.44 0.0
Germany female 10.13 0.0
Italy male 8.84 0.0003
Italy female 11.61 0.0
Morocco male 0.17 0.4548
Morocco female 11.41 0.0841


In [20]:
for country in list_countries:
    for gender in list_gender:
        sum_pears = []
        for k in range(n_counterfactuals):
            pears, p_val = pearsonr(proba_counter['negative'][country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals], 
                    list_PPL[country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals]
                   )
            sum_pears.append(pears)
        print(country, gender, np.round(100*np.mean(sum_pears), 2), np.round(np.mean(p_val), 4))

France male 6.53 0.0013
France female 20.43 0.0
United_Kingdom male 4.05 0.0707
United_Kingdom female 15.76 0.0002
Ireland male 10.29 0.0187
Ireland female 14.69 0.0001
Spain male 9.86 0.11
Spain female 1.52 0.2068
Germany male -5.96 0.0001
Germany female 8.47 0.0115
Italy male 10.66 0.0126
Italy female 12.98 0.0032
Morocco male 4.6 0.001
Morocco female -7.89 0.2928


In [21]:
for country in list_countries:
    for gender in list_gender:
        sum_pears = []
        for k in range(n_counterfactuals):
            pears, p_val = pearsonr(proba_counter['neutral'][country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals], 
                    list_PPL[country][gender][k*n_counterfactuals:(k+1)*n_counterfactuals]
                   )
            sum_pears.append(pears)
        print(country, gender, np.round(100*np.mean(sum_pears), 2), np.round(np.mean(p_val), 4))

France male -13.84 0.0
France female -12.96 0.0
United_Kingdom male -1.89 0.0268
United_Kingdom female -18.44 0.0002
Ireland male -10.31 0.0088
Ireland female -6.45 0.0
Spain male -1.37 0.0003
Spain female 1.11 0.0
Germany male -8.56 0.0
Germany female -5.63 0.0
Italy male -5.71 0.0004
Italy female -11.96 0.0
Morocco male -4.38 0.2773
Morocco female -6.03 0.1453


## Standardize, but looking at the whole

So you can say, that the 'local' has 10% of correlation  

In [38]:
from sklearn.preprocessing import StandardScaler
def rescale(l):
    scaler = StandardScaler()
    return [kk for ll in scaler.fit_transform([[k] for k in l]) for kk in ll]

In [56]:
proba_counter_standardize = {v: {ct : {gender: np.concatenate([rescale(proba_counter[v][ct][gender][k*n_counterfactuals:(k+1)*n_counterfactuals]) for k in range(n_counterfactuals)]) for gender in list_gender} for ct in list_countries} for v in proba_counter.keys()}
list_PPL_standardize = {ct : {gender: np.concatenate([rescale(list_PPL[ct][gender][k*n_counterfactuals:(k+1)*n_counterfactuals]) for k in range(n_counterfactuals)]) for gender in list_gender} for ct in list_countries}

In [66]:
proba_counter_standardize_country = {v: 
                                     {gender: np.concatenate([rescale(np.concatenate([proba_counter[v][ct][gender][k*n_counterfactuals:(k+1)*n_counterfactuals] for ct in list_countries])) for k in range(n_counterfactuals)]) for gender in list_gender
                                     } for v in proba_counter.keys()}
list_PPL_standardize_country = {gender: np.concatenate([rescale(np.concatenate([list_PPL[ct][gender][k*n_counterfactuals:(k+1)*n_counterfactuals] for ct in list_countries])) for k in range(n_counterfactuals)]) for gender in list_gender}

In [69]:
proba_string = 'positive'
print('\n'+proba_string)
for gender in list_gender:
    print(gender, pearsonr(proba_counter_standardize_country[proba_string][gender], 
        list_PPL_standardize_country[gender]))
    
proba_string = 'negative'
print('\n'+proba_string)
for gender in list_gender:
    print(gender, pearsonr(proba_counter_standardize_country[proba_string][gender], 
        list_PPL_standardize_country[gender]))
    
proba_string = 'neutral'
print('\n'+proba_string)
for gender in list_gender:
    print(gender, pearsonr(proba_counter_standardize_country[proba_string][gender], 
        list_PPL_standardize_country[gender]))


positive
male (0.10715937197607905, 7.312364868692595e-46)
female (0.08737521758206662, 5.205799642228486e-31)

negative
male (0.0638962943348084, 2.6571459624901772e-17)
female (0.08631620645864191, 2.666417472453386e-30)

neutral
male (-0.050173243529974085, 3.115521714390752e-11)
female (-0.046143128089332394, 1.0153470693587247e-09)


# Create the tables with the changes in the classes   

In [2]:
import pandas as pd
import pickle as pkl 
import numpy as np
from sklearn.metrics import confusion_matrix
path_data = '/home/barriva/data/PPL_Positivity/'
fn = 'biases_Eurotweets_English_val_without_line_return.tsv_clean_test.tsv'

list_countries = ['United_Kingdom','Ireland','United_States','Canada','Australia','New_Zealand',
                  'South_Africa','India','Germany','France','Spain','Italy','Portugal','Hungary',
                  'Poland','Turkey','Morocco']

In [14]:
df = pd.read_csv(path_data + "cardiffnlp_twitter-roberta-base-emotion/" + fn, sep='\t', header=[0,1], index_col=0)
print(df[df.columns[1:-3]].to_latex(index=False))

male                                                    \
                proba  joy optimism anger sadness    KL_sym KL_sym_mean   
France           0.08  1.0     -5.0  -3.0     4.0  0.043077    0.000126   
United_Kingdom  -1.95 -2.0      1.0   3.0     6.0  0.041359    0.000123   
Ireland         -1.86 -2.0     -0.0   1.0     7.0  0.041607    0.000125   
Spain           -0.26  2.0    -13.0  -0.0     3.0  0.041257    0.000125   
Germany         -0.90 -0.0     -5.0  -0.0     5.0  0.044504    0.000124   
Italy           -0.10  2.0    -15.0  -1.0     1.0  0.043128    0.000127   
Morocco        -15.93 -9.0    -33.0  60.0   -17.0  0.173815    0.000124   
India           -4.32 -2.0     -6.0   9.0     5.0  0.064005    0.000127   
Canada          -1.78 -2.0     -0.0   2.0     5.0  0.042096    0.000125   
Australia       -2.19 -2.0      1.0   3.0     7.0  0.042138    0.000122   
New_Zealand     -2.05 -2.0      3.0   3.0     5.0  0.041881    0.000124   
United_States   -2.16 -2.0     -1.0   4.0     7.0  0.042056    0.000126   
South_Africa    -1.49 -1.0      0.0   1.0     6.0  0.040667    0.000124   
Portugal        -0.45  2.0    -13.0   1.0    -0.0  0.042765    0.000126   
Hungary         -0.66  2.0    -17.0  -1.0     4.0  0.044651    0.000124   
Poland          -0.61  2.0    -14.0  -2.0     5.0  0.047516    0.000125   
Turkey          -2.18  0.0    -12.0   5.0     2.0  0.044985    0.000125   

                  n_sent_modified  
               Unnamed: 8_level_1  
France                        516  
United_Kingdom                516  
Ireland                       516  
Spain                         516  
Germany                       516  
Italy                         516  
Morocco                       516  
India                         516  
Canada                        516  
Australia                     516  
New_Zealand                   516  
United_States                 516  
South_Africa                  516  
Portugal                      516  
Hungary                       516  
Poland                        516  
Turkey                        516

In [6]:
df = pd.read_csv(path_data + "cardiffnlp_twitter-roberta-base-emotion/" + fn, sep='\t', header=[0,1], index_col=0).loc[list_countries]
print(df[df.columns[1:-3]].to_latex(index=True))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{4}{l}{male} \\
{} &  joy & optimism & anger & sadness \\
\midrule
United\_Kingdom & -2.1 &      0.6 &   2.7 &     6.4 \\
Ireland        & -1.6 &     -0.4 &   1.3 &     6.8 \\
United\_States  & -2.3 &     -0.5 &   4.0 &     6.5 \\
Canada         & -1.6 &     -0.2 &   2.3 &     5.0 \\
Australia      & -2.3 &      0.9 &   3.2 &     6.6 \\
New\_Zealand    & -2.3 &      2.9 &   2.9 &     5.2 \\
South\_Africa   & -1.5 &      0.4 &   1.0 &     6.1 \\
India          & -2.5 &     -6.1 &   8.7 &     5.0 \\
Germany        & -0.0 &     -4.8 &  -0.2 &     4.7 \\
France         &  0.9 &     -5.1 &  -2.5 &     3.8 \\
Spain          &  1.7 &    -13.0 &  -0.4 &     2.7 \\
Italy          &  2.5 &    -15.5 &  -0.9 &     1.5 \\
Portugal       &  1.9 &    -12.9 &   1.1 &    -0.4 \\
Hungary        &  2.4 &    -17.2 &  -1.4 &     4.0 \\
Poland         &  2.0 &    -13.7 &  -2.4 &     5.1 \\
Turkey         &  0.2 &    -11.9 &   4.8 &     1.7 \\
Morocco        &

/tmp/ipykernel_121/2634388552.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[df.columns[1:-3]].to_latex(index=True))


# Sentiment on XLM, 

### OLD ones with perturbed examples that I have not anymore 

In [4]:
df = pd.read_csv(path_data + "OLD_cardiffnlp_twitter-xlm-roberta-base-sentiment/" + 'all-genders_'+fn, sep='\t', header=[0,1], index_col=0).loc[list_countries]
print(df[df.columns[:-9]].to_latex(index=True))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{4}{l}{male} \\
{} & proba & negative & neutral & positive \\
\midrule
United\_Kingdom & -1.43 &      5.0 &     1.0 &     -5.0 \\
Ireland        & -1.47 &      6.0 &     1.0 &     -4.0 \\
United\_States  & -1.35 &      5.0 &     2.0 &     -5.0 \\
Canada         & -1.45 &      6.0 &     1.0 &     -5.0 \\
Australia      & -1.35 &      6.0 &     1.0 &     -5.0 \\
New\_Zealand    & -1.41 &      6.0 &     1.0 &     -5.0 \\
South\_Africa   & -1.61 &      6.0 &     1.0 &     -5.0 \\
India          & -2.66 &      8.0 &     0.0 &     -4.0 \\
Germany        & -2.14 &      6.0 &     1.0 &     -5.0 \\
France         & -1.58 &      8.0 &    -0.0 &     -4.0 \\
Spain          & -2.46 &      6.0 &     3.0 &     -7.0 \\
Italy          & -1.98 &      7.0 &     1.0 &     -5.0 \\
Portugal       & -2.31 &      7.0 &     2.0 &     -6.0 \\
Hungary        & -2.29 &      6.0 &     2.0 &     -6.0 \\
Poland         & -1.94 &      2.0 &     4.0 &     -6.0 \\
Turkey

/tmp/ipykernel_121/3128721133.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[df.columns[:-9]].to_latex(index=True))


In [36]:
df = pd.read_csv(path_data + "cardiffnlp_twitter-roberta-base-hate/" + fn, sep='\t', header=[0,1], index_col=0).loc[list_countries]
print(df[df.columns[:-3]].to_latex(index=True))

\begin{tabular}{lrrr}
\toprule
{} & \multicolumn{3}{l}{male} \\
{} & proba & non-hate &  hate \\
\midrule
United\_Kingdom &  0.64 &     -0.0 &  24.0 \\
Ireland        &  0.81 &     -0.0 &  21.0 \\
United\_States  &  0.64 &     -0.0 &  22.0 \\
Canada         &  0.74 &     -0.0 &  21.0 \\
Australia      &  0.64 &     -0.0 &  23.0 \\
New\_Zealand    &  0.64 &     -0.0 &  22.0 \\
South\_Africa   &  0.70 &     -0.0 &  22.0 \\
India          &  1.26 &     -0.0 &  10.0 \\
Germany        &  0.60 &     -0.0 &  19.0 \\
France         &  1.23 &     -0.0 &  10.0 \\
Spain          &  1.39 &     -0.0 &   6.0 \\
Italy          &  0.79 &     -0.0 &  12.0 \\
Portugal       &  1.35 &     -0.0 &  10.0 \\
Hungary        &  1.13 &     -0.0 &   6.0 \\
Poland         &  1.06 &     -0.0 &  10.0 \\
Turkey         &  1.05 &     -0.0 &   8.0 \\
Morocco        &  1.07 &     -0.0 &   2.0 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_723/3293327063.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[df.columns[:-3]].to_latex(index=True))


In [37]:
df = pd.read_csv(path_data + "cardiffnlp_twitter-roberta-base-offensive/" + fn, sep='\t', header=[0,1], index_col=0).loc[list_countries]
print(df[df.columns[:-3]].to_latex(index=True))

\begin{tabular}{lrrr}
\toprule
{} & \multicolumn{3}{l}{male} \\
{} & proba & non-offensive & offensive \\
\midrule
United\_Kingdom &  0.22 &          -0.4 &       4.8 \\
Ireland        &  0.21 &          -0.4 &       4.6 \\
United\_States  &  0.02 &          -0.5 &       6.1 \\
Canada         &  0.31 &          -0.4 &       4.5 \\
Australia      &  0.21 &          -0.3 &       4.3 \\
New\_Zealand    &  0.24 &          -0.4 &       4.8 \\
South\_Africa   &  0.26 &          -0.3 &       3.9 \\
India          &  0.52 &           0.1 &      -1.6 \\
Germany        & -0.02 &          -0.3 &       3.3 \\
France         &  1.27 &          -0.0 &       0.1 \\
Spain          &  0.24 &          -0.2 &       2.7 \\
Italy          & -0.30 &          -0.2 &       2.5 \\
Portugal       &  0.26 &          -0.1 &       1.8 \\
Hungary        &  0.73 &           0.2 &      -2.1 \\
Poland         &  0.56 &           0.1 &      -1.3 \\
Turkey         &  0.47 &           0.0 &      -0.3 \\
Morocco        & 

/tmp/ipykernel_723/3677985063.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[df.columns[:-3]].to_latex(index=True))
